In [1]:
import pandas as pd 
import joblib
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [2]:
df_titanic = pd.read_csv('../static/data/titanic_train.csv')
df_titanic.head(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,0,24.000000,0,0,8.85,7,3
1,0,3,1,29.699118,8,2,69.55,7,3
2,1,2,0,30.000000,0,0,12.35,7,2


In [3]:
""" X_train = df_titanic.drop('Survived',axis=1).values
y_train =  df_titanic['Survived'].values
X_train.shape, y_train.shape """

" X_train = df_titanic.drop('Survived',axis=1).values\ny_train =  df_titanic['Survived'].values\nX_train.shape, y_train.shape "

In [3]:
X_train = df_titanic.iloc[:, 1:].values
y_train = df_titanic.iloc[:, 0].values
X_train.shape, y_train.shape

((668, 8), (668,))

In [4]:
df_test = pd.read_csv('../static/data/titanic_test.csv')
df_test.head(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,0,20.0,0,0,8.6625,7,3
1,0,3,1,22.0,0,0,7.8958,7,3
2,0,3,1,33.0,0,0,7.8958,7,0


In [6]:
""" X_test = df_test.drop('Survived',axis=1).values
y_test = df_test['Survived'].values
X_test.shape, y_test.shape """

" X_test = df_test.drop('Survived',axis=1).values\ny_test = df_test['Survived'].values\nX_test.shape, y_test.shape "

In [5]:
X_test = df_test.iloc[:, 1:].values
y_test = df_test.iloc[:, 0].values
X_test.shape, y_test.shape

((223, 8), (223,))

In [8]:
""" scaler = MinMaxScaler()
scaler.fit(df_titanic.drop('Survived',axis=1)) """

" scaler = MinMaxScaler()\nscaler.fit(df_titanic.drop('Survived',axis=1)) "

In [6]:
scaler = MinMaxScaler()
scaler.fit(df_titanic.iloc[:, 1:])

MinMaxScaler()

In [7]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled.shape, X_test_scaled.shape

((668, 8), (223, 8))

In [8]:
joblib.dump(scaler, '../static/model/titanic_scaler.pkl')

['../static/model/titanic_scaler.pkl']

### Logistic Regression

In [9]:
lr_clf = LogisticRegression()
lr_clf.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [10]:
params = {
    # 'C': [1, 3, 5, 7, 10, 20]
    'C': [0.01, 0.03, 0.05, 0.08, 0.1, 0.2, 0.3, 0.4, 1 ]
}

In [11]:
grid_cv = GridSearchCV(lr_clf, param_grid=params, scoring='accuracy', cv=5)
grid_cv.fit(X_train_scaled, y_train)
print(f'최고 평균 정확도: {grid_cv.best_score_:.4f}')
print('최적 파라미터:', grid_cv.best_params_) # best_params_: 좋은 파라미터를 보여줌.

최고 평균 정확도: 0.8174
최적 파라미터: {'C': 0.05}


In [12]:
best_lr = grid_cv.best_estimator_ # best_estimator_ : 최적의 파라미터로 모델 생성
pred = best_lr.predict(X_test_scaled)
acc_lr = accuracy_score(y_test, pred)
print(f'평균 정확도 : {acc_lr:.4f}')

평균 정확도 : 0.8027


In [13]:
joblib.dump(best_lr, '../static/model/titanic_lr.pkl')

['../static/model/titanic_lr.pkl']

### Decision Tree

In [14]:
dt_clf = DecisionTreeClassifier()
dt_clf.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': 'deprecated',
 'random_state': None,
 'splitter': 'best'}

In [15]:
params = {
    'max_depth': [1, 3, 6, 8, 10],
    'min_samples_split': [1, 3, 6]
}

In [16]:
grid_cv = GridSearchCV(dt_clf, param_grid=params, scoring='accuracy', cv=5)
grid_cv.fit(X_train_scaled, y_train)
print(f'최고 평균 정확도: {grid_cv.best_score_:.4f}')
print('최적 파라미터:', grid_cv.best_params_) # best_params_: 좋은 파라미터를 보여줌.

최고 평균 정확도: 0.8189
최적 파라미터: {'max_depth': 3, 'min_samples_split': 3}


In [17]:
best_dt = grid_cv.best_estimator_ # best_estimator_ : 최적의 파라미터로 모델 생성
pred = best_dt.predict(X_test_scaled)
acc_dt = accuracy_score(y_test, pred)
print(f'평균 정확도 : {acc_dt:.4f}')

평균 정확도 : 0.7982


In [18]:
joblib.dump(best_dt, '../static/model/titanic_dt.pkl')

['../static/model/titanic_dt.pkl']

### SVM

In [19]:
sv_clf = SVC()
sv_clf.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [20]:
params = {
    # 'C': [90, 91, 92, 93, 94, 95]
     'C': [13, 14, 15, 16, 17, 92]
}

In [21]:
grid_cv = GridSearchCV(sv_clf, param_grid=params, scoring='accuracy', cv=5)
grid_cv.fit(X_train_scaled, y_train)
print(f'최고 평균 정확도: {grid_cv.best_score_:.4f}')
print('최적 파라미터:', grid_cv.best_params_) # best_params_: 좋은 파라미터를 보여줌.

최고 평균 정확도: 0.8309
최적 파라미터: {'C': 17}


In [22]:
best_sv = grid_cv.best_estimator_ # best_estimator_ : 최적의 파라미터로 모델 생성
pred = best_sv.predict(X_test_scaled)
acc_sv = accuracy_score(y_test, pred)
print(f'평균 정확도 : {acc_sv:.4f}')

평균 정확도 : 0.8072


In [23]:
joblib.dump(best_sv, '../static/model/titanic_sv.pkl')

['../static/model/titanic_sv.pkl']

### 랜덤 포레스트(Random Forest)

In [24]:
rf_clf = RandomForestClassifier()
rf_clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [25]:
params = {
    'max_depth': [1, 3, 6, 8, 10],
    'min_samples_split': [1, 3, 6]
}

In [26]:
grid_cv = GridSearchCV(rf_clf, param_grid=params, scoring='accuracy', cv=5)
grid_cv.fit(X_train_scaled, y_train)
print(f'최고 평균 정확도: {grid_cv.best_score_:.4f}')
print('최적 파라미터:', grid_cv.best_params_) # best_params_: 좋은 파라미터를 보여줌.

최고 평균 정확도: 0.8309
최적 파라미터: {'max_depth': 6, 'min_samples_split': 3}


In [27]:
best_rf = grid_cv.best_estimator_ # best_estimator_ : 최적의 파라미터로 모델 생성
pred = best_rf.predict(X_test_scaled)
acc_rf = accuracy_score(y_test, pred)
print(f'평균 정확도 : {acc_rf:.4f}')

평균 정확도 : 0.8341


In [28]:
joblib.dump(best_rf, '../static/model/titanic_rf.pkl')

['../static/model/titanic_rf.pkl']

### K 최근접 이웃(K Nearest Neighbor)

In [29]:
kn_clf = KNeighborsClassifier()
kn_clf.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [30]:
params = {
    'n_neighbors': [1, 2, 5, 6, 7]
}

In [31]:
grid_cv = GridSearchCV(kn_clf, param_grid=params, scoring='accuracy', cv=5)
grid_cv.fit(X_train_scaled, y_train)
print(f'최고 평균 정확도: {grid_cv.best_score_:.4f}')
print('최적 파라미터:', grid_cv.best_params_) # best_params_: 좋은 파라미터를 보여줌.

최고 평균 정확도: 0.8159
최적 파라미터: {'n_neighbors': 6}


In [32]:
best_kn = grid_cv.best_estimator_ # best_estimator_ : 최적의 파라미터로 모델 생성
pred = best_kn.predict(X_test_scaled)
acc_kn = accuracy_score(y_test, pred)
print(f'평균 정확도 : {acc_kn:.4f}')

평균 정확도 : 0.8161


In [33]:
joblib.dump(best_kn, '../static/model/titanic_kn.pkl')

['../static/model/titanic_kn.pkl']

### Test

In [34]:
index = 100

In [35]:
test_data = (df_test.iloc[index, 1:].values).reshape(1,-1)
label = df_test.iloc[index, 0]
test_data, label

(array([[ 2. ,  0. , 29. ,  0. ,  0. , 10.5,  5. ,  3. ]]), 1)

In [36]:
new_scaler = joblib.load('../static/model/titanic_scaler.pkl')

In [37]:
test_scaled = new_scaler.transform(test_data)

In [38]:
""" pred_lr = best_lr.predict(test_data)
pred_sv = best_sv.predict(test_data)
pred_dt = best_dt.predict(test_data)
pred_rf = best_rf.predict(test_data)
pred_kn = best_kn.predict(test_data) """

' pred_lr = best_lr.predict(test_data)\npred_sv = best_sv.predict(test_data)\npred_dt = best_dt.predict(test_data)\npred_rf = best_rf.predict(test_data)\npred_kn = best_kn.predict(test_data) '

In [39]:
pred_lr = best_lr.predict(test_scaled)
pred_sv = best_sv.predict(test_scaled)
pred_dt = best_dt.predict(test_scaled)
pred_rf = best_rf.predict(test_scaled)
pred_kn = best_kn.predict(test_scaled)

In [40]:
label, pred_lr[0], pred_sv[0], pred_dt[0], pred_rf[0], pred_kn[0]

(1, 1, 1, 1, 1, 1)

In [41]:
titanic_dict = {'label':label, 'pred_lr':pred_lr[0], 'area_ratio': pred_sv[0], 'per_person':pred_dt[0], 'pred_rf': pred_rf[0], 'pred_kn': pred_kn[0]}
titanic_dict

{'label': 1,
 'pred_lr': 1,
 'area_ratio': 1,
 'per_person': 1,
 'pred_rf': 1,
 'pred_kn': 1}

In [42]:
 titanic_dict = {'label':label, 'pred_lr':pred_lr[0], 'pred_sv':pred_sv[0], 'pred_dt':pred_dt[0], 'pred_rf': pred_rf[0], 'pred_kn': pred_kn[0]}

In [43]:
titanic_dict

{'label': 1,
 'pred_lr': 1,
 'pred_sv': 1,
 'pred_dt': 1,
 'pred_rf': 1,
 'pred_kn': 1}